In [1]:
import cuquantum


/opt/conda/lib/python3.8/site-packages/cupy/_environment.py:439: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda11x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [2]:
import numpy as np
import cupy as cp

import cuquantum
from cuquantum import custatevec as cusv


nIndexBits = 3
nSvSize    = (1 << nIndexBits)
nMaxShots  = 5
nShots     = 5

bitStringLen  = 2;
bitOrdering   = np.asarray([0, 1], dtype=np.int32)

bitStrings = np.empty((nShots,), dtype=np.int64)
bitStrings_expected = np.asarray([0b00, 0b01, 0b10, 0b11, 0b11], dtype=np.int64)

h_sv          = np.asarray([0.0+0.0j, 0.0+0.1j, 0.1+0.1j, 0.1+0.2j,
                            0.2+0.2j, 0.3+0.3j, 0.3+0.4j, 0.4+0.5j], dtype=np.complex64)

d_sv = cp.asarray(h_sv)

# In real appliction, random numbers in range [0, 1) will be used.
randnums      = np.asarray([0.1, 0.8, 0.4, 0.6, 0.2], dtype=np.float64)

########################################################################

# cuStateVec handle initialization
handle = cusv.create()

# create sampler and check the size of external workspace
sampler, extraWorkspaceSizeInBytes = cusv.sampler_create(
    handle, d_sv.data.ptr, cuquantum.cudaDataType.CUDA_C_32F, nIndexBits, nMaxShots)

# allocate external workspace
extraWorkspace = cp.cuda.alloc(extraWorkspaceSizeInBytes)

# sample preprocess
cusv.sampler_preprocess(
    handle, sampler, extraWorkspace.ptr, extraWorkspaceSizeInBytes)

# sample bit strings
cusv.sampler_sample(
    handle, sampler, bitStrings.ctypes.data, bitOrdering.ctypes.data, bitStringLen,
    randnums.ctypes.data, nShots, cusv.SamplerOutput.ASCENDING_ORDER)

# destroy sampler
cusv.sampler_destroy(sampler)

# destroy handle
cusv.destroy(handle)

if not np.allclose(bitStrings, bitStrings_expected):
    raise ValueError("results mismatch")
print("test passed")

test passed


In [3]:
import numpy as np
import cupy as cp
from cuquantum import custatevec as cusv
from cuquantum import cudaDataType as cudtype
from cuquantum import ComputeType as ctype

# Initialize parameters
nIndexBits = 2  # 2 qubits
nSvSize = (1 << nIndexBits)
adjoint = 0

# Qubit indices (corrected for cuQuantum indexing)
h_target = (1,)  # Apply Hadamard to the second qubit (which is qubit 1 in zero-based index)
cnot_targets = (1,)  # Target qubit for CNOT (qubit 0)
cnot_controls = (0,)  # Control qubit for CNOT (qubit 1)

# Initial statevector |00>
d_sv = cp.zeros(nSvSize, dtype=cp.complex128)
d_sv[0] = 1.0 + 0.0j  # Setting |00> state

# Define Hadamard gate (correct as complex64)
h_matrix = cp.array([[1, 1], [1, -1]], dtype=cp.complex128) / np.sqrt(2)

# Define CNOT gate (correct as complex64)
cnot_matrix = cp.array([[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 0, 1],
                        [0, 0, 1, 0]], dtype=cp.complex128)

# cuStateVec handle initialization
handle = cusv.create()

# Apply Hadamard gate to the second qubit
cusv.apply_matrix(
    handle, d_sv.data.ptr, cudtype.CUDA_C_64F, nIndexBits,
    h_matrix.data.ptr, cudtype.CUDA_C_64F, cusv.MatrixLayout.ROW, adjoint,
    h_target, len(h_target), (), 0, 0, ctype.COMPUTE_64F,
    0, 0)

# Debug: print state vector after applying Hadamard gate to the second qubit
print("State vector after Hadamard gate to the second qubit:")
print(d_sv)

# Verify intermediate state vector manually
expected_after_hadamard = (1 / np.sqrt(2)) * cp.array([1.0, 0.0, 1.0, 0.0], dtype=cp.complex64)
print("Expected state vector after Hadamard gate to the second qubit:")
print(expected_after_hadamard)
assert cp.allclose(d_sv, expected_after_hadamard), "Hadamard gate application failed."

# Apply CNOT gate with qubit 1 as control and qubit 0 as target
cusv.apply_matrix(
    handle, d_sv.data.ptr, cudtype.CUDA_C_64F, nIndexBits,
    cnot_matrix.data.ptr, cudtype.CUDA_C_64F, cusv.MatrixLayout.ROW, adjoint,
    cnot_targets, len(cnot_targets), cnot_controls, 0, len(cnot_controls), ctype.COMPUTE_64F,
    0, 0)

# Destroy handle
cusv.destroy(handle)

# Debug: print state vector after applying CNOT gate
print("State vector after CNOT gate:")
print(d_sv)
# Expected Bell state (|00> + |11>) / sqrt(2)
bell_state = (1 / np.sqrt(2)) * cp.array([1.0, 0.0, 0.0, 1.0], dtype=cp.complex128)

# Check if d_sv holds the Bell state
correct = cp.allclose(d_sv, bell_state)
if not correct:
    raise RuntimeError("example FAILED: wrong result")

print("Bell state creation successful.")
print("Resulting state vector:", d_sv)

State vector after Hadamard gate to the second qubit:
[0.70710678+0.j 0.        +0.j 0.70710678+0.j 0.        +0.j]
Expected state vector after Hadamard gate to the second qubit:
[0.70710677+0.j 0.        +0.j 0.70710677+0.j 0.        +0.j]
State vector after CNOT gate:
[0.70710678+0.j 0.        +0.j 0.70710678+0.j 0.        +0.j]


RuntimeError: example FAILED: wrong result